In [1]:
import os
from pathlib import Path
import pandas as pd
import QuantLib as ql
import numpy as np
from datetime import datetime
from model_settings import ms
from itertools import product
file_dir = os.path.join(str(Path().resolve()),'historical_data_collection','historical_av_collection')
files = os.listdir(file_dir)
filename = [f for f in files if f.endswith('.h5')][0]

file = os.path.join(file_dir,filename)
with pd.HDFStore(file) as store:
    df = store["/date_2022_10_20/heston_calibration/calibration_results"]
eg = df.iloc[0]
eg


pricing settings:
Actual/365 (Fixed) day counter
New York stock exchange calendar
compounding: continuous
frequency: annual



strike_price             335.0
w                         call
market_price              34.6
volatility             0.33141
days_to_maturity            34
calculation_date    2022-10-20
spot_price              365.41
risk_free_rate            0.04
dividend_rate              0.0
theta                  0.17771
kappa                 0.412367
eta                   0.785592
rho                  -0.582856
v0                     0.08079
moneyness             0.090776
black_scholes        35.227613
heston_price          34.77996
error                -0.012707
Name: 3420, dtype: object

In [2]:
past_fixings = 0
rng = "pseudorandom" # could use "lowdiscrepancy"
numPaths = 100000


def asian_option_price(s,k,r,g,w,averaging_type,n_fixings,fixing_frequency,kappa,theta,rho,eta,v0,calculation_datetime):
    if w == 'call':
        option_type = ql.Option.Call 
    elif w == 'put':
        option_type = ql.Option.Put
    t = n_fixings*fixing_frequency
    
    if isinstance(calculation_datetime, np.datetime64):
        calculation_datetime = calculation_datetime.astype('M8[ms]').astype(datetime)
    
    calculation_date = ql.Date(calculation_datetime.day,calculation_datetime.month,calculation_datetime.year)
    
    periods = np.arange(fixing_frequency,t+1,fixing_frequency).astype(int)
    
    fixing_dates = [calculation_date + ql.Period(int(p),ql.Days) for p in periods]
    expiration_date = calculation_date + ql.Period(int(t),ql.Days)
    
    riskFreeTS = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, float(r), ql.Actual365Fixed()))
    dividendTS = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, float(g), ql.Actual365Fixed()))
    
    hestonProcess = ql.HestonProcess(riskFreeTS, dividendTS, ql.QuoteHandle(ql.SimpleQuote(s)), v0, kappa, theta, eta, rho)
    hestonModel = ql.HestonModel(hestonProcess)
    vanillaPayoff = ql.PlainVanillaPayoff(option_type, float(k))
    europeanExercise = ql.EuropeanExercise(expiration_date)
    
    if averaging_type == 'geometric':
        geometric_engine = ql.MCDiscreteGeometricAPHestonEngine(hestonProcess, rng, requiredSamples=numPaths,seed=123)
        geometricAverage = ql.Average().Geometric
        geometricRunningAccumulator = 1.0
        discreteGeometricAsianOption = ql.DiscreteAveragingAsianOption(
        	geometricAverage, geometricRunningAccumulator, past_fixings,
            fixing_dates, vanillaPayoff, europeanExercise)
        discreteGeometricAsianOption.setPricingEngine(geometric_engine)
        geometric_price = float(discreteGeometricAsianOption.NPV())
        return geometric_price
        
    elif averaging_type == 'arithmetic':
        arithmetic_engine = ql.MCDiscreteArithmeticAPHestonEngine(hestonProcess, rng, requiredSamples=numPaths)
        arithmeticAverage = ql.Average().Arithmetic
        arithmeticRunningAccumulator = 0.0
        discreteArithmeticAsianOption = ql.DiscreteAveragingAsianOption(
            arithmeticAverage, arithmeticRunningAccumulator, past_fixings, 
            fixing_dates, vanillaPayoff, europeanExercise)
        discreteArithmeticAsianOption.setPricingEngine(arithmetic_engine)
        arithmetic_price = float(discreteArithmeticAsianOption.NPV())
        return arithmetic_price
    else:
        print("invalid Asian option averaging type out of 'arithmetic' and geometric'")
        pass

In [3]:
vector_asian_price = np.vectorize(asian_option_price)
calculation_datetime = datetime.today()

r = 0.04
g = 0.0

w = 'put'


s = 365.41
k = 370.00
kappa = 0.412367
theta = 0.17771
rho = -0.582856
eta = 0.785592
v0 = 0.08079
n_fixings = 5
fixing_frequency = 50

averaging_type = 'geometric'

t = n_fixings*fixing_frequency


price = asian_option_price(s,k,r,g,w,averaging_type,n_fixings,fixing_frequency,kappa,theta,rho,eta,v0,calculation_datetime)

vanilla = ms.ql_heston_price(s,k,t,r,g,w,kappa,theta,rho,eta,v0,datetime.today())

print(f"{price}\n{vanilla}")

21.715962339603674
28.766454289828687


In [4]:
def vector_asian_price(df):
    v_asian = np.vectorize(asian_option_price)
    prices = v_asian(
        df['spot_price'],
        df['strike_price'],
        df['risk_free_rate'],
        df['dividend_rate'],
        df['w'],
        df['averaging_type'],
        df['n_fixings'],
        df['fixing_frequency'],
        df['kappa'],
        df['theta'],
        df['rho'],
        df['eta'],
        df['v0'],
        df['calculation_date']
    )
    return prices

In [11]:
fixing_frequency = [1,7,30,90]
n_fixings = [5,10]
K = np.unique(np.linspace(s*0.5,s*1.5,3).astype(int))

features = pd.DataFrame(
    product(
        [s],
        K,
        [r],
        [g],
        fixing_frequency,
        n_fixings,
        ['call','put'],
        ['geometric','arithmetic'],
        [theta],
        [kappa],
        [rho],
        [eta],
        [v0],
        [calculation_datetime]
    ),
    columns = [
        'spot_price','strike_price',
        'risk_free_rate','dividend_rate',
        'fixing_frequency','n_fixings','w','averaging_type',
        'theta','kappa','rho','eta','v0','calculation_date'
    ]
)
features['days_to_maturity'] = features['n_fixings']*features['fixing_frequency']
features.loc[:,'vanilla_price'] = ms.vector_heston_price(features)
features

,spot_price,strike_price,risk_free_rate,dividend_rate,fixing_frequency,n_fixings,w,averaging_type,theta,kappa,rho,eta,v0,calculation_date,days_to_maturity,vanilla_price
0,365.41,182,0.04,0.0,1,5,call,geometric,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 22:14:04.465270,5,183.509699
1,365.41,182,0.04,0.0,1,5,call,arithmetic,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 22:14:04.465270,5,183.509699
2,365.41,182,0.04,0.0,1,5,put,geometric,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 22:14:04.465270,5,0.000000
3,365.41,182,0.04,0.0,1,5,put,arithmetic,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 22:14:04.465270,5,0.000000
4,365.41,182,0.04,0.0,1,10,call,geometric,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 22:14:04.465270,10,183.609343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,365.41,548,0.04,0.0,90,5,put,arithmetic,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 22:14:04.465270,450,160.353605
92,365.41,548,0.04,0.0,90,10,call,geometric,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 22:14:04.465270,900,16.209814
93,365.41,548,0.04,0.0,90,10,call,arithmetic,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 22:14:04.465270,900,16.209814
94,365.41,548,0.04,0.0,90,10,put,geometric,0.17771,0.412367,-0.582856,0.785592,0.08079,2024-10-17 22:14:04.465270,900,147.330432


In [12]:
features.loc[:,'asian_price'] = vector_asian_price(features)

E:\Python\Lib\site-packages\numpy\lib\_function_base_impl.py:2553: RuntimeWarning: overflow encountered in asian_option_price (vectorized)
  outputs = ufunc(*inputs)


In [13]:
features.describe()

,spot_price,strike_price,risk_free_rate,dividend_rate,fixing_frequency,n_fixings,theta,kappa,rho,eta,v0,calculation_date,days_to_maturity,vanilla_price,asian_price
count,9.600000e+01,96.000000,9.600000e+01,96.0,96.000000,96.000000,96.00000,96.000000,9.600000e+01,96.000000,9.600000e+01,96,96.000000,96.000000,96.000000
mean,3.654100e+02,365.000000,4.000000e-02,0.0,32.000000,7.500000,0.17771,0.412367,-5.828560e-01,0.785592,8.079000e-02,2024-10-17 22:14:04.465270528,240.000000,69.530972,65.209071
min,3.654100e+02,182.000000,4.000000e-02,0.0,1.000000,5.000000,0.17771,0.412367,-5.828560e-01,0.785592,8.079000e-02,2024-10-17 22:14:04.465270,5.000000,0.000000,0.000000
25%,3.654100e+02,182.000000,4.000000e-02,0.0,5.500000,5.000000,0.17771,0.412367,-5.828560e-01,0.785592,8.079000e-02,2024-10-17 22:14:04.465270016,28.750000,2.918220,0.313098
50%,3.654100e+02,365.000000,4.000000e-02,0.0,18.500000,7.500000,0.17771,0.412367,-5.828560e-01,0.785592,8.079000e-02,2024-10-17 22:14:04.465270016,110.000000,20.577678,13.490042
75%,3.654100e+02,548.000000,4.000000e-02,0.0,45.000000,10.000000,0.17771,0.412367,-5.828560e-01,0.785592,8.079000e-02,2024-10-17 22:14:04.465270016,337.500000,174.983485,177.805401
max,3.654100e+02,548.000000,4.000000e-02,0.0,90.000000,10.000000,0.17771,0.412367,-5.828560e-01,0.785592,8.079000e-02,2024-10-17 22:14:04.465270,900.000000,208.436305,186.299119
std,1.142836e-13,150.203231,6.975319e-18,0.0,35.377067,2.513123,0.00000,0.000000,1.116051e-16,0.000000,1.395064e-17,NaN,291.012751,80.757625,81.394275
